In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from feedback import KMeansSegmentation, VectorSegmenter, HyperplaneSegmentation
from funcs import get_kxk_square, get_kxk_square_values, assess_square, signal_results, row_agg, hash, kmeans_step, plane_step, initialize_segmenters
from classifiers3x3 import c0, c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13

# Create vector of zeros length 364, to be modified when learning starts
kmeans_amps = np.zeros(364)
plane_amps = np.zeros(364)

test = pd.read_csv('digit-recognizer/test.csv')
train = pd.read_csv('digit-recognizer/train.csv')
labels = train.iloc[:, 0] 
data = train.iloc[:, 1:]  

In [124]:
# Set variables for the image, kxk square size, and starting index
k = 5
w, h = 28, 28
p = 98

# Select an image from the dataset
image = 1937

# Get the column indeces for the kxk square, then get the specific values for the selected image
pix = get_kxk_square(p = p, k = k)
values_list = get_kxk_square_values(r = image, k = k, pix = pix, data = data)

# Print as a little matrix
print(values_list)


[[  0   0   0   0   0]
 [  0   0   0   0   0]
 [144 173 254 148  65]
 [196 253 253 253 237]
 [  6  44  44 202 253]]


In [125]:
# Assess interesting parts of the square
## avg_value, dark_cols, dark_rows, light_cols, light_rows, is_interesting
signals = assess_square(values_list = values_list)
print(signals)

(101.0, False, True, False, True, True)


In [126]:
k = len(values_list)
avg = signals[0]
light_threshold = avg - 40
light_mask = values_list <= light_threshold

if (np.any(np.all(light_mask[:,n+1:m+1], axis=0) for m in range(k-1) for n in range(k-1) if m > n)):
    print("True")
else: print("False")

True


In [129]:
print(light_mask)
print("---------------------------")

if (
    (
        any(np.all(light_mask[:, :n], axis=0) for n in range(2))         # Dark left
        and
        any(np.all(light_mask[:, -(n+1):], axis=0) for n in range(3))   # Dark right
    )
    or
    any(np.any(np.all(light_mask[:, n+1:m+1], axis=0)) for n in range(k-1) for m in range(n+1, k))    # Light middle
):
    print(True)
else: print(False)

[[ True  True  True  True  True]
 [ True  True  True  True  True]
 [False False False False False]
 [False False False False False]
 [ True  True  True False False]]
---------------------------
False


/var/folders/w6/j8d1_lqx5wz_zztjb28h8rkh0000gn/T/ipykernel_8482/3404266777.py:6: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  any(np.all(light_mask[:, :n], axis=0) for n in range(2))         # Dark left


In [4]:
############## Empty/Full ##############
print(f"Empty sensor returns: {c0(signals=signals)}")
print(f"Full sensor returns: {c1(signals=signals)}")
############## Columns ##############
print(f"Left dark vbar sensor returns: {c2(values_list=values_list, signals=signals)}")
print(f"Right light vbar sensor returns: {c3(values_list=values_list, signals=signals)}")
print(f"Middle light vbar sensor returns: {c4(values_list=values_list, signals=signals)}")
print(f"Middle dark vbar sensor returns: {c5(values_list=values_list, signals=signals)}")
print(f"Left light vbar sensor returns: {c6(values_list=values_list, signals=signals)}")
print(f"Right dark vbar sensor returns: {c7(values_list=values_list, signals=signals)}")
############## Rows ##############
print(f"Top dark hbar sensor returns: {c8(values_list=values_list, signals=signals)}")
print(f"Bottom light hbar sensor returns: {c9(values_list=values_list, signals=signals)}")
print(f"Middle light hbar sensor returns: {c10(values_list=values_list, signals=signals)}")
print(f"Middle dark hbar sensor returns: {c11(values_list=values_list, signals=signals)}")
print(f"Top light hbar sensor returns: {c12(values_list=values_list, signals=signals)}")
print(f"Bottom dark hbar sensor returns: {c13(values_list=values_list, signals=signals)}")

Empty sensor returns: False
Full sensor returns: False
Left dark vbar sensor returns: False
Right light vbar sensor returns: True
Middle light vbar sensor returns: False
Middle dark vbar sensor returns: False
Left light vbar sensor returns: False
Right dark vbar sensor returns: False
Top dark hbar sensor returns: False
Bottom light hbar sensor returns: True
Middle light hbar sensor returns: False
Middle dark hbar sensor returns: False
Top light hbar sensor returns: False
Bottom dark hbar sensor returns: False


In [42]:
k = 14
kleft = np.floor(k/2).astype(int)
kright = np.ceil(k/2).astype(int)
print([kleft+n for n in range(kright-1)])
print(kleft, kright)

[7, 8, 9, 10, 11, 12]
7 7


In [70]:
k = 12
v = [k for k in range(k)]
print(v, '\n---------------------')
for n in range(k-1):
    for m in range(k-1):
        if m>n:
            print(v[n+1:m+1])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11] 
---------------------
[1]
[1, 2]
[1, 2, 3]
[1, 2, 3, 4]
[1, 2, 3, 4, 5]
[1, 2, 3, 4, 5, 6]
[1, 2, 3, 4, 5, 6, 7]
[1, 2, 3, 4, 5, 6, 7, 8]
[1, 2, 3, 4, 5, 6, 7, 8, 9]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[2]
[2, 3]
[2, 3, 4]
[2, 3, 4, 5]
[2, 3, 4, 5, 6]
[2, 3, 4, 5, 6, 7]
[2, 3, 4, 5, 6, 7, 8]
[2, 3, 4, 5, 6, 7, 8, 9]
[2, 3, 4, 5, 6, 7, 8, 9, 10]
[3]
[3, 4]
[3, 4, 5]
[3, 4, 5, 6]
[3, 4, 5, 6, 7]
[3, 4, 5, 6, 7, 8]
[3, 4, 5, 6, 7, 8, 9]
[3, 4, 5, 6, 7, 8, 9, 10]
[4]
[4, 5]
[4, 5, 6]
[4, 5, 6, 7]
[4, 5, 6, 7, 8]
[4, 5, 6, 7, 8, 9]
[4, 5, 6, 7, 8, 9, 10]
[5]
[5, 6]
[5, 6, 7]
[5, 6, 7, 8]
[5, 6, 7, 8, 9]
[5, 6, 7, 8, 9, 10]
[6]
[6, 7]
[6, 7, 8]
[6, 7, 8, 9]
[6, 7, 8, 9, 10]
[7]
[7, 8]
[7, 8, 9]
[7, 8, 9, 10]
[8]
[8, 9]
[8, 9, 10]
[9]
[9, 10]
[10]


In [5]:
n = 784
image = 1937

binary_matrix = signal_results(values_list=values_list, data=data, image=image)
# print(binary_matrix)

In [6]:
result_vector = row_agg(binary_matrix=binary_matrix)
# print("Result vector:", result_vector)
# print(len(result_vector))

In [7]:
kmeans_output = result_vector + kmeans_amps
plane_output = result_vector + plane_amps
# print(output)
# print(len(output))

In [8]:
kmeans_segmenter, plane_segmenter = initialize_segmenters(num_sections=11, len=364)